In [1]:
# imports
import xgboost as xgb
from sklearn.model_selection import train_test_split
import pandas as pd

In [2]:
# read and format training data
x_training_data = pd.read_csv('data/train.csv')
y_training_data = x_training_data['Survived']
x_training_data = x_training_data.drop(columns=['Survived', 'Name', 'Ticket', 'Cabin', 'PassengerId'])

x_training_data['Sex'] = x_training_data['Sex'].map({'male': 0, 'female': 1})
x_training_data['Embarked'] = x_training_data['Embarked'].map({'S': 0, 'C': 1, 'Q': 2})

xgtrain = xgb.DMatrix(x_training_data, y_training_data)

# read and format testing data

test_data = pd.read_csv('data/test.csv')
test_data = test_data.drop(columns=['Name', 'Ticket', 'Cabin', 'PassengerId'])

test_data['Sex'] = test_data['Sex'].map({'male': 0, 'female': 1})
test_data['Embarked'] = test_data['Embarked'].map({'S': 0, 'C': 1, 'Q': 2})

xgtest = xgb.DMatrix(test_data)

In [16]:
# create and train model
params = {
    'objective': 'binary:logistic',
    'max_depth': 3,
    'learning_rate': 0.1,
    'eval_metric': 'error'
}

num_round = 1000

model = xgb.train(params, xgtrain, num_round)

In [18]:
# predict
predictions = model.predict(xgtest)
predictions = [1 if x > 0.5 else 0 for x in predictions]

# write to file
output = pd.read_csv('data/test.csv')
output = output[['PassengerId']]
output['Survived'] = predictions
output.to_csv('output.csv', index=False)